## 데이터 불러오기

In [5]:
import pandas as pd
df_seoul120 = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df_seoul120

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492
4,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798
...,...,...,...,...,...
2640,5,주택도시계획,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...,2894353
2641,4,환경,수돗물의 톤당 원가는?,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다,19353862
2642,3,복지,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...,19699584
2643,2,교통,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...,2898293


In [6]:
import pandas as pd
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## 자연어 전처리

In [222]:
# stopwords = ["하다","돋다","있다","되다","이다","의하다"]
stopwords = ["있다","이다","길음동","서울시"]

In [223]:
# 토크나이징 함수 정의
from konlpy.tag import Okt
okt = Okt()
def Tokenizer(raw, pos=["Noun","Modifier","Adjective"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [224]:
okt.pos(df_seoul120["내용"][2290])

[('수화', 'Noun'),
 ('통역', 'Noun'),
 ('센터', 'Noun'),
 ('운영', 'Noun'),
 ('안내', 'Noun'),
 ('업무', 'Noun'),
 ('개요', 'Noun'),
 ('각', 'Noun'),
 ('지역별', 'Noun'),
 ('수화', 'Noun'),
 ('통역', 'Noun'),
 ('센터', 'Noun'),
 ('안내', 'Noun'),
 ('업무', 'Noun'),
 ('설명', 'Noun'),
 ('수화', 'Noun'),
 ('통역', 'Noun'),
 ('센터', 'Noun'),
 ('지역', 'Noun'),
 ('시', 'Modifier'),
 ('설명', 'Noun'),
 ('소재지', 'Noun'),
 ('전화번호', 'Noun'),
 ('영상전화', 'Noun'),
 ('번호', 'Noun'),
 ('강남구', 'Noun'),
 ('강남구', 'Noun'),
 ('수화', 'Noun'),
 ('통역', 'Noun'),
 ('센터', 'Noun'),
 ('강남구', 'Noun'),
 ('개포로', 'Noun'),
 ('길', 'Noun'),
 ('개포동', 'Noun'),
 ('대화', 'Noun'),
 ('빌딩', 'Noun'),
 ('호', 'Noun'),
 ('강북구', 'Noun'),
 ('강북구', 'Noun'),
 ('수화', 'Noun'),
 ('통역', 'Noun'),
 ('센터', 'Noun'),
 ('강북구', 'Noun'),
 ('한천로', 'Noun'),
 ('길', 'Noun'),
 ('번동', 'Noun'),
 ('경', 'Modifier'),
 ('일', 'Modifier'),
 ('빌딩', 'Noun'),
 ('층', 'Noun'),
 ('호', 'Noun'),
 ('강동구', 'Noun'),
 ('강동구', 'Noun'),
 ('수화', 'Noun'),
 ('통역', 'Noun'),
 ('센터', 'Noun'),
 ('강동구', 'Noun'),
 ('풍성로', '

In [225]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer = Tokenizer, use_idf=True,max_df=0.95,min_df=2)

In [226]:
# # tokenizing
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidfVectorizer = TfidfVectorizer(use_idf=True,max_df=0.95,min_df=2)

In [227]:
features = tfidfVectorizer.fit_transform(df_seoul120["내용"])
features.toarray()[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [236]:
import numpy as np 
np.where(features.toarray()[:1] > 0 )

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]),
 array([  64,   69,  202,  205,  318,  339,  359,  369,  392,  447,  743,
         747,  763,  884,  931,  936,  999, 1078, 1106, 1339, 1348, 1587,
        1756, 1848, 2038, 2048, 2110, 2121, 2127, 2210, 2312, 2357, 2398,
        2426, 2502, 2710, 2881, 2899, 2927, 2928, 3075, 3530, 3622, 3716,
        3723, 3779, 3992, 4015, 4285, 4332, 4420, 4433, 4447, 4506, 4575,
        4612, 4704, 4717, 4783, 4817, 5010, 5104, 5239, 5286, 5331, 5332,
        5468, 5522, 5523, 5841, 6115, 6284, 6631, 6687]))

In [237]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=5, random_state=111) #인스턴스화 #n_components 토픽의 갯수
lda_model.fit(features) #교육

LatentDirichletAllocation(n_components=5, random_state=111)

In [238]:
dictionary_list = tfidfVectorizer.get_feature_names_out()

In [239]:
dictionary_list

array(['가가', '가게', '가격', ..., '힐링', '힐스테이트', '힘들다'], dtype=object)

In [240]:
lda_model.components_

array([[0.2001133 , 0.20007823, 0.20052151, ..., 0.20002626, 0.20006657,
        0.2001733 ],
       [0.20011271, 0.20007425, 0.20006904, ..., 0.20002561, 0.20006672,
        0.20016615],
       [0.41232217, 0.33528693, 3.7956023 , ..., 1.04873458, 0.20668308,
        0.52676041],
       [0.20009614, 0.20006744, 0.24545644, ..., 0.20002238, 0.20007568,
        0.20015322],
       [0.69346013, 0.200052  , 4.59734371, ..., 0.20000874, 0.48474488,
        0.30757315]])

In [241]:
# list(zip(dictionary_list, lda_model.components_[0],lda_model.components_[1],lda_model.components_[2])) # concaterate/ zip은 1차원 행렬만 가능

In [242]:
df_topics

,0,1
405,39.923152,계획
5368,37.45395,주택
1417,32.615702,도로
273,31.908262,건축물
2676,30.70265,사업
...,...,...
5567,0.200008,직능단체
6150,0.200008,특강
4228,0.200006,위신
2960,0.200006,선조


In [243]:
import pandas as pd
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add = [['Topic0', 'Topic1', 'Topic2','Topic3','Topic4'],topics_list]
df_topics_keywords = pd.DataFrame(topics_list_add)
df_topics_keywords=df_topics_keywords.T
df_topics_keywords

,0,1
0,Topic0,임기 인사 위원회 자격정지 위원 특별회계 가요 이첩 명인 백서
1,Topic1,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
2,Topic2,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
3,Topic3,디자인 조합 등기 의원 임원 재결 몇개 설립 입찰 택시
4,Topic4,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지


In [244]:
df_topics_keywords.loc[1,1]

'자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목'

In [245]:
topics_output = lda_model.transform(features)
topics_output # 토픽별 점수

array([[0.02636255, 0.02639449, 0.89315284, 0.02637287, 0.02771725],
       [0.04077924, 0.0407892 , 0.8366585 , 0.04078635, 0.04098672],
       [0.03294424, 0.03286873, 0.40610334, 0.03287829, 0.4952054 ],
       ...,
       [0.03813919, 0.03814034, 0.84699709, 0.03829041, 0.03843298],
       [0.04803548, 0.04914615, 0.04991982, 0.04806677, 0.80483178],
       [0.05493511, 0.05515585, 0.77926651, 0.05494231, 0.05570021]])

In [246]:
df_topics_score = pd.DataFrame(topics_output)
df_topics_score

,0,1,2,3,4
0,0.026363,0.026394,0.893153,0.026373,0.027717
1,0.040779,0.040789,0.836658,0.040786,0.040987
2,0.032944,0.032869,0.406103,0.032878,0.495205
3,0.025721,0.025711,0.896255,0.026306,0.026007
4,0.029595,0.029611,0.881406,0.029608,0.029780
...,...,...,...,...,...
2640,0.035600,0.035635,0.036896,0.035595,0.856274
2641,0.069633,0.071427,0.717162,0.069683,0.072095
2642,0.038139,0.038140,0.846997,0.038290,0.038433
2643,0.048035,0.049146,0.049920,0.048067,0.804832


In [247]:
import numpy as np
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)

In [248]:
df_topics_score

,0,1,2,3,4,dominant_topic_number
0,0.026363,0.026394,0.893153,0.026373,0.027717,2
1,0.040779,0.040789,0.836658,0.040786,0.040987,2
2,0.032944,0.032869,0.406103,0.032878,0.495205,4
3,0.025721,0.025711,0.896255,0.026306,0.026007,2
4,0.029595,0.029611,0.881406,0.029608,0.029780,2
...,...,...,...,...,...,...
2640,0.035600,0.035635,0.036896,0.035595,0.856274,4
2641,0.069633,0.071427,0.717162,0.069683,0.072095,2
2642,0.038139,0.038140,0.846997,0.038290,0.038433,2
2643,0.048035,0.049146,0.049920,0.048067,0.804832,4


In [249]:
df_topics_score['sentences']=df_seoul120["내용"]

In [250]:
df_topics_score

,0,1,2,3,4,dominant_topic_number,sentences
0,0.026363,0.026394,0.893153,0.026373,0.027717,2,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...
1,0.040779,0.040789,0.836658,0.040786,0.040987,2,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...
2,0.032944,0.032869,0.406103,0.032878,0.495205,4,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...
3,0.025721,0.025711,0.896255,0.026306,0.026007,2,광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...
4,0.029595,0.029611,0.881406,0.029608,0.029780,2,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...
...,...,...,...,...,...,...,...
2640,0.035600,0.035635,0.036896,0.035595,0.856274,4,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...
2641,0.069633,0.071427,0.717162,0.069683,0.072095,2,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다
2642,0.038139,0.038140,0.846997,0.038290,0.038433,2,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...
2643,0.048035,0.049146,0.049920,0.048067,0.804832,4,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...


In [299]:
index_0 = df_topics_score.query('dominant_topic_number==0').index
index_1 = df_topics_score.query('dominant_topic_number==1').index
index_2 = df_topics_score.query('dominant_topic_number==2').index
index_3 = df_topics_score.query('dominant_topic_number==3').index
index_4 = df_topics_score.query('dominant_topic_number==4').index

In [300]:
for i in range(len(df_topics_score)):
    if i in index_0:    
        df_topics_score.loc[i,"topic_keywords"]= df_topics_keywords.loc[0,1]
    elif i in index_1:
        df_topics_score.loc[i,"topic_keywords"]= df_topics_keywords.loc[1,1]
    elif i in index_2:
        df_topics_score.loc[i,"topic_keywords"]= df_topics_keywords.loc[2,1]
    elif i in index_3:
        df_topics_score.loc[i,"topic_keywords"]= df_topics_keywords.loc[3,1]
    elif i in index_4:
        df_topics_score.loc[i,"topic_keywords"]= df_topics_keywords.loc[4,1]

### topic_1 : 건설, 토목으로 예상
topic keyword가 자본금, 기술, 건설, 토목 등으로 구성되어 있는 것으로 보아 건설이나 토목과 관련된 민원으로 예상된다.
문장들을 봤을 때 기술능력이나 자본금에 대한 민원들이 많은 것을 알 수 있다. 사업별 기술능력과 자본금에 대한 정보를 제공해주는 서비스를 제공해주면 관련 민원들을 많이 해결할 수 있을 것 같다.

In [301]:
df_topics_score[["dominant_topic_number","sentences","topic_keywords"]].query("dominant_topic_number==1")[:20]

,dominant_topic_number,sentences,topic_keywords
437,1,장지천 조명등 수명은 친환경조명등인 는 수명이 일반조명등에 비하여 상당히 긴것이 장...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
450,1,행정대집행 계고란이행기간을 정하여 그 기한까지 이행되지 않으면 대집행을 한다는 뜻을...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
650,1,거실의 외벽이란 거실의 벽중에서 외기에 직간접적으로 면하는 부위를 말합니다,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
651,1,외피란거실 또는 거실을 둘러싸고 있는 벽 지붕 바닥 창 및 문으로써 외기에 직접 면...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
670,1,시설물유지관리업의 기술능력과 자본금은 어떻게 되나요건설기술관리법에 의한 토목분야 또...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
672,1,난방시공업제종의 기술능력과 자본금은 어떻게 되나요 돋...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
682,1,준설공사업의 기술능력과 자본금은 어떻게 되나요 건설기술관리법에 의한 토목분야의 토목...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
685,1,삭도설치공사업의 기술능력과 자본금은 어떻게 되나요 건설기술관리법에 의한 기계 토목 ...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
686,1,철강재설치공사업의 기술능력과 자본금은 어떻게 되나요 건설기술관리법에 의한 토목분야의...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목
688,1,강구조물공사업의 기술능력과 자본금은 어떻게 되나요 건설기술관리법에 의한 토목 건축 ...,자본금 기술 건설 이상 기술자 의하다 토목 국가기술자격법 분야 종목


### toic_2 : 복지, 교육으로 예상
topic keyword가 운영, 신청, 센터, 교육 등으로 구성되어 있는 것으로 보아 나라나 지자체에서 운영하는 복지 프로그램이나 복지 시설에 관련된 민원으로 예상된다.
이 데이터들을 통해 요즘 사람들이 어느 복지 프로그램에 관심이 많고, 불만이 많은지 파악할 수 있을 것으로 보인다.

In [302]:
df_topics_score[["dominant_topic_number","sentences","topic_keywords"]].query("dominant_topic_number==2")[:20]

,dominant_topic_number,sentences,topic_keywords
0,2,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
1,2,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
3,2,광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
4,2,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
5,2,년도 중고 신입생 입학준비금 지원 업무개요 서울시는 전국 최초로 년도부터 개 자...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
6,2,위드유 서울 직장 성희롱성폭력 예방센터 업무개요 직장 내 성희롱 지원체계 사각지...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
7,2,성동구 반려동물 무료상담실 운영 안내 반려동물 무료상담실무엇이든 물어보시개 운영 안...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
8,2,서울시 우리동네꼼꼼육아정보 책자서울시와 서울시육아종합지원센터는 여 명의 우리동네 보...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
9,2,반반택시 심야시간 택시 동승앱 업무개요 심야 택시수요 대비 택시공급 부족으로 발...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육
10,2,서울시 사회서비스원은 무엇을 하는 곳인가요 사회서비스 분야에서 공공부문의 역할을 강...,사업 운영 민원 경우 신청 센터 공무원 업무 지원 교육


### topic_4 : 
topic keyword가 주택, 도로, 건축물, 도시 건축, 토지 등으로 구성되어 있는 것으로 보아 도로, 건축물에 관련된 민원으로 예상된다.
이 데이터들을 통해 부동산이나 도로와 관련된 정책 중 어떤 사항을 개선해야할지 파악할 수 있을 것으로 보인다.

In [303]:
df_topics_score[["dominant_topic_number","sentences","topic_keywords"]].query("dominant_topic_number==4")[:20]

,dominant_topic_number,sentences,topic_keywords
2,4,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
19,4,사업이란 뭔가요란 에너지절약전문기업의 약자로 에너지사용자가 에너지절약을 위하여 기...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
20,4,사업의 주요 사업분야에는 어떤게 있나요사업 분야에는 산업체 건물의 소형 열병합 발전...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
21,4,에너지절약전문기업사업의 장점은에너지사용자는 기술적 또는 경제적 부담없이 에너지절약형...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
40,4,주택 소방시설 설치 의무화주택용 소방시설 설치 의무화업무개요 실제로 화재가 많이나고...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
62,4,달라지는 업무 고시원 및 숙박시설 활용 저렴주택공급 구 분 내 용 ...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
68,4,태양광 시민펀드 시민들의 투자금을 모아 태양광 발전소를 짓고 전기를 판매해 수익을 ...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
136,4,건축물대장에 소매점이라고 되어있는데 일반음식점 영업하고자 할 때 표시변경 신청을 해...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
151,4,두레주택셰어하우스업무개요 한 지붕 아래 다수의 인 가구가 함께 어울려 살되 각자의 ...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
153,4,명동삼일대로 지하 관광버스 전용주차장 조성 업무개요 서울시 명동 지하에 ...,계획 주택 도로 건축물 사업 시설 도시 경우 건축 토지
